<a href="https://colab.research.google.com/github/SuyashGupta123/NITDAUDITIONS2025/blob/main/logsolubilityai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install deepchem

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 17.4 MB/s eta 0:00:00


In [2]:
%pip install --upgrade deepchem

In [3]:
%pip install --upgrade numpy scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 92.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.3 which is incompatible.
pytensor 2.27.1 requires

In [4]:
%pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0

In [3]:
import deepchem as dc
import numpy as np
import torch

tasks, datasets, transformers = dc.molnet.load_delaney(
    featurizer='ECFP', splitter='scaffold'
)
train_dataset, valid_dataset, test_dataset = datasets

model = dc.models.MultitaskRegressor(
    n_tasks=len(tasks), n_features=1024, layer_sizes=[200, 100]
)

model.fit(train_dataset, nb_epoch=20)

metric = dc.metrics.Metric(dc.metrics.rms_score, mode='regression')

print('Training set score:')
train_scores = model.evaluate(train_dataset, [metric], transformers)
print(train_scores)

print('Validation set score:')
valid_scores = model.evaluate(valid_dataset, [metric], transformers)
print(valid_scores)

print('Test set score:')
test_scores = model.evaluate(test_dataset, [metric], transformers)
print(test_scores)

def predict_delaney(smiles_string, model, featurizer):
    features = featurizer.featurize(smiles_string)

    if features.size == 0:
        return "Failed to featurize SMILES."

    features = np.expand_dims(features, axis=0)
    prediction = model.predict_on_batch(features)

    return prediction[0]

smiles = 'CCO'
featurizer = dc.feat.CircularFingerprint(size=1024)
prediction = predict_delaney(smiles, model, featurizer)
print(f'Prediction for {smiles}: {prediction}')

# Save the model state dictionary (PyTorch)
torch.save(model.model.state_dict(), 'delaney_model_weights.pth')

# Load the model state dictionary (PyTorch)
loaded_model = dc.models.MultitaskRegressor(
    n_tasks=len(tasks), n_features=1024, layer_sizes=[200, 100]
)

# Build the model by predicting on dummy data.
dummy_features = np.zeros((1, 1024))
loaded_model.predict_on_batch(dummy_features)

loaded_model.model.load_state_dict(torch.load('delaney_model_weights.pth'))

loaded_prediction = predict_delaney(smiles, loaded_model, featurizer)
print(f'Prediction using loaded model for {smiles}: {loaded_prediction}')

Training set score:
{'rms_score': np.float64(0.8100692423225221)}
Validation set score:
{'rms_score': np.float64(1.783301060308124)}
Test set score:
{'rms_score': np.float64(1.7968917769582757)}
Prediction for CCO: [[1.1156521]]
Prediction using loaded model for CCO: [[1.1156521]]


[05:04:48] DEPRECATION WARNING: please use MorganGenerator
<ipython-input-3-781d777b1ce5>:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.model.load_state_dic

In [19]:
import deepchem as dc
import numpy as np
import torch

# 1. Load and Prepare the Dataset (Delaney - Regression)

tasks, datasets, transformers = dc.molnet.load_delaney(
    featurizer='ECFP', splitter='scaffold'
)
train_dataset, valid_dataset, test_dataset = datasets

# 2. Model Selection and Training (MultitaskRegressor)

model = dc.models.MultitaskRegressor(
    n_tasks=len(tasks), n_features=1024, layer_sizes=[200, 100]
)

model.fit(train_dataset, nb_epoch=20)

# 3. Save the Model Weights (PyTorch)
torch.save(model.model.state_dict(), 'delaney_model_weights.pth')

# 4. Prediction Function (User Input)

def predict_delaney_user_input(smiles_string, model_path='delaney_model_weights.pth'):
    """Predicts log solubility for a user-provided SMILES string."""

    # Load the model state dictionary (PyTorch)
    loaded_model = dc.models.MultitaskRegressor(
        n_tasks=1, n_features=1024, layer_sizes=[200, 100] #n_tasks = 1 because delaney has one task.
    )

    # Build the model by predicting on dummy data.
    dummy_features = np.zeros((1, 1024))
    loaded_model.predict_on_batch(dummy_features)

    try:
        loaded_model.model.load_state_dict(torch.load(model_path, weights_only=True))
    except FileNotFoundError:
        return "Model weights file not found."

    featurizer = dc.feat.CircularFingerprint(size=1024)
    features = featurizer.featurize(smiles_string)

    if features.size == 0:
        return "Failed to featurize SMILES."

    features = np.expand_dims(features, axis=0)
    prediction = loaded_model.predict_on_batch(features)

    return prediction[0]

# 5. User Input and Prediction

user_smiles = input("Enter SMILES string: ")
prediction = predict_delaney_user_input(user_smiles)

if isinstance(prediction, str):
    print(prediction)  # Print error messages
else:
    print(f"Predicted log solubility: {prediction}")

Enter SMILES string: CCCCCC
Predicted log solubility: [[-0.35519537]]


[05:43:40] DEPRECATION WARNING: please use MorganGenerator
